In [30]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import gurobipy as gp
from gurobipy import GRB

In [31]:
macrotick = 100
sync_error = 0
time_out = 4 * 60 * 60

NUM_FLOW = 1000
DATA_NAME = "orion0"
TOPO_NAME = "3"

task = pd.read_csv("../../dac_data/%s.csv"%DATA_NAME)[:NUM_FLOW]
network = pd.read_csv("../../dac_data/%s_topology.csv"%TOPO_NAME)
for col in ['size','period','deadline','jitter']:
    task[col] = np.ceil(task[col] / macrotick).astype(int)
for col in ['t_proc','t_prop']:
    network[col] = np.ceil(network[col] / macrotick).astype(int)
    
nodes = list(network['link'].apply(lambda x:eval(x)[0])) + \
    list(network['link'].apply(lambda x:eval(x)[1]))
NODE_SET = list(set(nodes))
ES_set = [x for x in NODE_SET if nodes.count(x) == 2]
SW_set = list(set(NODE_SET) - set(ES_set))
LCM = np.lcm.reduce(task['period'])
net = np.zeros(shape = (max(NODE_SET) + 1, max(NODE_SET) + 1))

In [32]:
M = int(1e16)

In [33]:
task_attr = {}
for k, row in task.iterrows():
#     task_var.setdefault(k, {})
    task_attr.setdefault(k, {})
    
    ## f_k
    task_attr[k]['src'] = int(row['src'])
    task_attr[k]['dst'] = int(eval(row['dst'])[0])
    task_attr[k]['cycle_time'] = int(row['period'])
    task_attr[k]['size'] = int(row['size'])
    task_attr[k]['l'] = int(row['deadline'])
    task_attr[k]['dtr'] = int(row['size']) * 8
    
#     ## Bound the t matrix
#     for j in index_to_link:
#         m.addConstr(t[k][j] <= task_attr[k]['ct'] - 1)


In [34]:
net_var = {}
for _, row in network.iterrows():
    net_var.setdefault(eval(row['link'])[0], {})
    net_var[eval(row['link'])[0]]['msd'] = row['t_proc']
    net[eval(row['link'])[0], eval(row['link'])[1]] = 1

## Create mapping from Link to index
link_to_index = {}
index_to_link = {}

counter = 0
for _, row in network.iterrows():
    link = row['link']
    link_to_index[link] = counter
    index_to_link[counter] = link
    counter += 1

## Preprocessing

In [35]:
def find_all_paths(graph, start, end, path=[]):
    path = path + [start]
    if start == end:
        return [path]
    paths = []
    for node in set(np.reshape(np.argwhere(graph[start] > 0),  -1)):
        if node not in path:
            newpaths = find_all_paths(graph, node, end, path)
            for newpath in newpaths:
                paths.append(newpath)
    return paths     

In [36]:
paths = {}
for i in task_attr:
    paths[i] = find_all_paths(net, task_attr[i]['src'],task_attr[i]['dst'])
    for k in range(len(paths[i])):
        paths[i][k] = list({x: int(eval(str(paths[i][k]))[h+1]) for h, x in enumerate(eval(str(paths[i][k]))[:-1])}.items())

In [37]:
route_space = {}
for i in paths:
    route_space[i] = set([str(x) for y in paths[i] for x in y if len(y) * (task_attr[i]['size'] + max(network['t_proc'])) <= task_attr[i]['l']])
    # route_space[i] = set(link_to_index.keys())

## 1. Model

In [38]:
m = gp.Model("RTNS2021")

Network model

$$\mathcal{G}=\{\mathcal{V}, \mathcal{E}\}$$

$$s=\left(v_{\text {src }}, v_{\mathrm{dst}}, \text { size }, l\right) \mid v_{s}, v_{d} \in \text { Hosts, } l \in(0, \text { cycle_time }], \text { size } \in[64, M T U]$$

Add route mapping

In [39]:
link_in = {}
link_out = {}
for link in link_to_index.keys():
    link = eval(link)
    link_in.setdefault(link[1], [])
    link_in[link[1]].append(str(link))
    link_out.setdefault(link[0], [])
    link_out[link[0]].append(str(link))

In [40]:
x = m.addMVar(shape=(len(task), len(link_to_index)), vtype=GRB.BINARY , name="routing")
start = m.addMVar(shape=(len(task), len(link_to_index)), vtype=GRB.INTEGER , name="time_start")
end = m.addMVar(shape=(len(task), len(link_to_index)), vtype=GRB.INTEGER , name="time_end")

## 2. Constraints

### 2.1 Route constraints

$$\forall\left(v_{\mathrm{src}}, v_{\mathrm{dst}}, \mathrm{size}, d, c\right) \in \mathcal{S}: \sum_{e \in \text { in_edges }\left(v_{\mathrm{src}}\right)} \mathrm{x}_{\mathrm{s}, \mathrm{e}}=0$$

In [41]:
for s in task_attr:
    m.addConstr(
        gp.quicksum(x[s][link_to_index[link]] for link in link_in[task_attr[s]['src']]
                   if link in route_space[s]) == 0
    )

$$\forall\left(v_{\mathrm{src}}, v_{\mathrm{dst}}, \mathrm{size}, d, c\right) \in \mathcal{S}: \sum_{e \in \text { out_edges }\left(v_{\mathrm{src}}\right)} \mathrm{x}_{\mathrm{s}, \mathrm{e}}=1$$

In [42]:
for s in task_attr:
    m.addConstr(
        gp.quicksum(x[s][link_to_index[link]] for link in link_out[task_attr[s]['src']]
                   if link in route_space[s]) == 1
    )
    ### Have to specify the source
    for v in ES_set:
        m.addConstr(
            gp.quicksum(
                   x[s][link_to_index[link]] for link in link_out[v] if v != task_attr[s]['src']
                and link in route_space[s]
            ) == 0
        )

$$\forall\left(v_{\mathrm{src}}, v_{\mathrm{dst}}, \mathrm{size}, d, c\right) \in \mathcal{S}: \sum_{\left.e \in \text { out_edges }{ }_{\mathrm{dst}}\right)} \mathrm{x}_{\mathrm{s}, \mathrm{e}}=0$$

In [43]:
for s in task_attr:
    m.addConstr(
        gp.quicksum(x[s][link_to_index[link]] for link in link_out[task_attr[s]['dst']]
                   if link in route_space[s]) == 0
    )

$$\forall\left(v_{\mathrm{src}}, v_{\mathrm{dst}}, \mathrm{size}, d, c\right) \in \mathcal{S}: \sum_{e \in \text { in_edges }\left(v_{\mathrm{dst}}\right)} \mathrm{x}_{\mathrm{s}, \mathrm{e}}=1$$

In [44]:
for s in task_attr:
    m.addConstr(
        gp.quicksum(x[s][link_to_index[link]] for link in link_in[task_attr[s]['dst']]
                   if link in route_space[s]) == 1
    )

$$\begin{gathered}
\forall\left(v_{\mathrm{src}}, v_{\mathrm{dst}}, \mathrm{size}, d, c\right) \in \mathcal{S}, \forall v \in \mathcal{V} \backslash\left\{v_{\mathrm{src}}, v_{\mathrm{dst}}\right\}: \\
\sum_{e \in \text { in_edges }(v)} \mathrm{x}_{\mathrm{s}, \mathrm{e}}=\sum_{e \in \text { out_edges }(v)} \mathrm{x}_{\mathrm{s}, \mathrm{e}}
\end{gathered}$$

In [45]:
for s in task_attr:
    for v in SW_set:
        m.addConstr(
            gp.quicksum(x[s][link_to_index[link]] for link in link_in[v]
                       if link in route_space[s])
            ==
            gp.quicksum(x[s][link_to_index[link]] for link in link_out[v]
                       if link in route_space[s])
        )

\begin{gathered}
\forall\left(v_{\mathrm{src}}, v_{\mathrm{dst}}, \operatorname{size}, d, c\right) \in \mathcal{S}, \forall v \in \mathcal{V} \backslash\left\{v_{\mathrm{src}}, v_{\mathrm{dst}}\right\}: \\
\sum_{e \in \text { out_edges }(v)} \mathrm{X}_{\mathrm{s}, \mathrm{e}} \leq 1
\end{gathered}

In [46]:
for s in task_attr:
    for v in SW_set:
        m.addConstr(
            gp.quicksum(x[s][link_to_index[link]] for link in link_out[v]
                       if link in route_space[s]) <= 1
        )

### 2.2 Scheduling constraint

$$\forall e \in \mathcal{E}, \forall s \in \mathcal{S}: \text { end }_{\mathrm{s}, \mathrm{e}} \leq c y c l e_{-} \text {time } \cdot \mathrm{x}_{\mathrm{s}, \mathrm{e}}$$

In [47]:
for s in task_attr:
    for e in index_to_link:
        if index_to_link[e] in route_space[s]:
            m.addConstr(
                end[s][e] <= task_attr[s]['cycle_time'] * x[s][e]
            )

\begin{gathered}
\forall e \in \mathcal{E}, \forall s \in \mathcal{S}: \mathrm{end}_{\mathrm{s}, \mathrm{e}}=\mathrm{start}_{\mathrm{s}, \mathrm{e}}+ \\
\mathrm{x}_{\mathrm{s}, \mathrm{e}} \cdot \text { trans_delay }(\text { stream_size,link_speed }(e))
\end{gathered}

In [48]:
for s in task_attr:
    for e in index_to_link:
        if index_to_link[e] in route_space[s]:
            m.addConstr(
                end[s][e] == start[s][e] + x[s][e] * task_attr[s]['dtr']
            )
        

\begin{aligned}
&\forall\left(v_{\text {src }}, v_{\mathrm{dst}}, \text { size }, d, c\right) \in \mathcal{S}, \forall v \in \mathcal{V} \backslash\left\{v_{\text {src }}, v_{\mathrm{dst}}\right\}\\
&\sum_{e \in \text { in_edges }(v)} \text { end }_{\mathrm{s}, \mathrm{e}}+\mathrm{x}_{\mathrm{s}, \mathrm{e}} \cdot\left(\text { prop_delay }(e)+\operatorname{proc}_{-} \operatorname{delay}(v)\right)\\
&=\sum_{e \in \text { out_edges }(v)} \operatorname{start}_{\mathrm{s}, \mathrm{e}}
\end{aligned}

In [49]:
for s in task_attr:
    for v in SW_set:
        m.addConstr(
            gp.quicksum(
                end[s][link_to_index[e]] +  x[s][link_to_index[e]] * net_var[eval(e)[1]]['msd']
                for e in link_in[v] if e in route_space[s]
            ) == 
            gp.quicksum(
                start[s][link_to_index[e]]
                for e in link_out[v] if e in route_space[s]
            )
        )

⚠️ Only for cut-through switches

\begin{aligned}
&\forall\left(v_{\mathrm{src}}, v_{\mathrm{dst}}, \mathrm{size}, d, c\right) \in \mathcal{S}, \forall v \in \mathcal{V} \backslash\left\{v_{\mathrm{src}}, v_{\mathrm{dst}}\right\}:\\
&\sum_{e \in \text { in_edges }(v)} \text { start }_{\mathrm{s}, \mathrm{e}}+\text { trans_delay }\left(H E A D E R_{-} \text {SIZE,link_speed }(e)\right)
\end{aligned}

In [50]:
# for s in task_attr:
#     for v in range(NUM_SW):
#         m.addConstr(
#             gp.quicksum(
#                 start[s][e] + task_attr[s]['dtr']
#             )
#         )

\begin{gathered}
\forall\left(\left(s, s^{\prime}\right), e\right) \in(\mathcal{S} \times \mathcal{S}) \times \mathcal{E} \mid s \neq s^{\prime}: \\
\text { end }_{\mathrm{s}, \mathrm{e}} \leq \mathrm{start} \mathrm{s}_{\mathrm{s}^{\prime}, \mathrm{e}}+\mathrm{y}_{\mathrm{s}, \mathrm{s}^{\prime}, \mathrm{e}} \cdot M \\
\text { end }_{\mathrm{s}^{\prime}, \mathrm{e}} \leq \mathrm{star} \mathrm{t}_{\mathrm{s}, \mathrm{e}}+\left(1-\mathrm{y}_{\mathrm{s}, \mathrm{s}^{\prime}, \mathrm{e}}\right) \cdot M
\end{gathered}

In [51]:
for s, s_p in tqdm([(s, s_p) for s in task_attr for s_p in task_attr if s < s_p]):
    s_t, s_p_t = task.loc[s].period, task.loc[s_p].period
    lcm = np.lcm(s_t, s_p_t)
    for e in index_to_link:
        if index_to_link[e] in route_space[s] and index_to_link[e] in route_space[s_p]:
            for a, b in [(a,b) for a in range(0, int(lcm / s_t)) for b in range(0, int(lcm / s_p_t))]:
                _inte = m.addVar(vtype= GRB.BINARY, name = "%d%d%s"%(s, s_p, index_to_link[e]))
                m.addConstr(
                    end[s][e] + a * s_t <= start[s_p][e] - 1 + b * s_p_t + (2 + _inte - x[s][e] - x[s_p][e]) * M
                )
                m.addConstr(
                    end[s_p][e] + b * s_p_t <= start[s][e] - 1 + a * s_t + (3 - _inte - x[s][e] - x[s_p][e]) * M
                )

100%|██████████| 17391/17391 [11:56<00:00, 24.27it/s] 


In [52]:
try: 
    m.optimize()
except gp.GurobiError as E:
    print("Optimize failed", E)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 14406180 rows, 7251832 columns and 71961962 nonzeros
Model fingerprint: 0x63fcaf9d
Variable types: 0 continuous, 7251832 integer (7209944 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+16]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+16]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 6633 rows and 0 columns (presolve time = 5s) ...
Presolve removed 8782 rows and 43761 columns (presolve time = 12s) ...
Presolve removed 2127849 rows and 43762 columns (presolve time = 16s) ...
Presolve removed 4246905 rows and 2162818 columns (presolve time = 20s) ...
Presolve removed 4246905 rows and 2162818 columns (presolve time = 32s) ...
Presolve removed 4246905 rows and 2162818 columns (presolve time =

: 

: 

## Output Schedule

In [ ]:
## GCL
GCL = []
for i in task_attr:
    period = task.loc[i, 'period']
    for e_i in index_to_link:
        link = index_to_link[e_i]
        if x[i][e_i].x > 0:
            s = start[i][e_i].x
            e = end[i][e_i].x
            queue = 0
            for k in range(int(LCM / period)):
                GCL.append(
                    [eval(link), 0, int(s + k * period) * macrotick, int(e + k * period) * macrotick, LCM * macrotick]
                )

In [ ]:
## Offset
OFFSET = []
for i in task_attr:
    start_link = [link for link in link_out[task_attr[i]['src']] if x[i][link_to_index[link]].x > 0][0]
    offset = start[i, link_to_index[start_link]].x
    OFFSET.append(
        [i, 0, (task.loc[i,'period'] - offset) * macrotick]
    )    

In [ ]:
ROUTE = []
for i in task_attr:
    for k, rr in enumerate(x[i]):
        if rr.x > 0:
            ROUTE.append(
                [i, eval(index_to_link[k])]
            )

In [ ]:
QUEUE = []
for i in task_attr:
    for k, rr in enumerate(x[i]):
        if rr.x > 0:
            e = index_to_link[k]
            QUEUE.append([i, 0, eval(e), 0])

In [ ]:
GCL = pd.DataFrame(GCL)
GCL.columns = ["link", "queue", "start", "end", "cycle"]
GCL.to_csv("RTNS2021-%s-%d-%s-GCL.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

OFFSET = pd.DataFrame(OFFSET)
OFFSET.columns = ['id', 'ins_id', 'offset']
OFFSET.to_csv("RTNS2021-%s-%d-%s-OFFSET.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

ROUTE = pd.DataFrame(ROUTE)
ROUTE.columns = ['id', 'link']
ROUTE.to_csv("RTNS2021-%s-%d-%s-ROUTE.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

QUEUE = pd.DataFrame(QUEUE)
QUEUE.columns = ['id','ins_id','link','queue']
QUEUE.to_csv("RTNS2021-%s-%d-%s-QUEUE.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)